In [1]:
!pip install google-search-results


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Mustafa Ansari\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
from serpapi import GoogleSearch
import pandas as pd
from datetime import datetime, timedelta
import time
import re

# ✅ Step 1: Your SerpAPI Key
API_KEY = "--"

# ✅ Step 2: Parse Relative/Absolute Dates
def parse_relative_date(text):
    today = datetime.today()
    try:
        if re.search(r"\w{3,} \d{1,2}, \d{4}", text):  # e.g. Apr 4, 2025
            return datetime.strptime(text, "%b %d, %Y")
        elif "day" in text:
            days = int(re.search(r"(\d+)", text).group(1))
            return today - timedelta(days=days)
        elif "hour" in text:
            hours = int(re.search(r"(\d+)", text).group(1))
            return today - timedelta(hours=hours)
        elif "week" in text:
            weeks = int(re.search(r"(\d+)", text).group(1))
            return today - timedelta(weeks=weeks)
        elif "month" in text:
            months = int(re.search(r"(\d+)", text).group(1))
            return today - timedelta(days=30 * months)
        else:
            return None
    except:
        return None

# ✅ Step 3: Scraper Function
def scrape_serpapi_news(company, target_articles=600):
    print(f"🔍 Scraping {company}...")
    results = []
    page = 0
    collected = 0
    today = datetime.today()
    ninety_days_ago = today - timedelta(days=90)

    while collected < target_articles:
        params = {
            "engine": "google",
            "q": f"{company} stock",
            "tbm": "nws",
            "api_key": API_KEY,
            "num": 100,
            "start": page * 100,
            "tbs": "cdr:1,cd_min:1/14/2024,cd_max:4/14/2025"
        }

        search = GoogleSearch(params)
        response = search.get_dict()
        news_results = response.get("news_results", [])

        if not news_results:
            print(f"⚠️ No more news results found at page {page} for {company}")
            break

        for article in news_results:
            pub_date_raw = article.get("date")
            parsed_date = parse_relative_date(pub_date_raw)

            if parsed_date and parsed_date >= ninety_days_ago:
                results.append({
                    "company": company,
                    "title": article.get("title"),
                    "link": article.get("link"),
                    "snippet": article.get("snippet"),
                    "source": article.get("source"),
                    "published": parsed_date.strftime("%m/%d/%Y")
                })

        collected = len(results)
        page += 1
        time.sleep(1)

    print(f"✅ Collected {len(results)} valid articles for {company}")
    return pd.DataFrame(results[:target_articles])

# ✅ Step 4: List of Companies
companies = ["Apple", "Tesla", "Nvidia", "Amazon"]

# ✅ Step 5: Scrape & Save
all_dfs = []
for company in companies:
    df = scrape_serpapi_news(company, target_articles=600)
    all_dfs.append(df)

final_df = pd.concat(all_dfs, ignore_index=True)


🔍 Scraping Apple...
⚠️ No more news results found at page 7 for Apple
✅ Collected 446 valid articles for Apple
🔍 Scraping Tesla...
✅ Collected 604 valid articles for Tesla
🔍 Scraping Nvidia...
⚠️ No more news results found at page 7 for Nvidia
✅ Collected 502 valid articles for Nvidia
🔍 Scraping Amazon...
⚠️ No more news results found at page 7 for Amazon
✅ Collected 362 valid articles for Amazon
📁 Done! Saved to: serpapi_google_news_601_last90days.csv
company
Tesla     600
Nvidia    502
Apple     446
Amazon    362
Name: count, dtype: int64


In [5]:
final_df 

,company,title,link,snippet,source,published
0,Apple,Apple Joins AI Data Center Race After Siri Mess,https://www.investors.com/news/technology/appl...,Apple is in the process of placing orders for ...,Investor's Business Daily,03/25/2025
1,Apple,Where Will Apple Stock Be In 5 Years?,https://www.forbes.com/sites/investor-hub/arti...,The shares currently trade approximately 12% b...,Forbes,03/16/2025
2,Apple,Apple stock rout shaves nearly $700 billion fr...,https://finance.yahoo.com/news/apple-stock-rou...,In This Article: AAPL · GOOG. Apple (AAPL) sto...,Yahoo Finance,03/16/2025
3,Apple,Analysts revisit Apple stock price targets as ...,https://www.thestreet.com/investing/analysts-r...,"Wedbush analyst Dan Ives, a committed Apple bu...",TheStreet,03/25/2025
4,Apple,How Bad Could Sustained Tariffs Be for Apple S...,https://www.morningstar.com/stocks/how-bad-cou...,"If tariffs persist, Apple's profit margins cou...",Morningstar,04/10/2025
...,...,...,...,...,...,...
1905,Amazon,Amazon Stock Down Pre-Market on April 11 After...,https://hindustanherald.com/amazon-stock-april...,Amazon stock closed flat at $181.22 on April 1...,Hindustan Herald,04/12/2025
1906,Amazon,Amazon Stock Pre-Market Drops 2.22% After Powe...,https://hindustanherald.com/amazon-stock-pre-m...,Amazon stock pre-market falls 2.22% to $186.86...,Hindustan Herald,04/10/2025
1907,Amazon,Amazon Stock Faces Early Dip to $184.35 on Apr...,https://hindustanherald.com/amazon-stock-april...,"Seattle/New York, April 14 (IST): Amazon stock...",Hindustan Herald,04/14/2025
1908,Amazon,Amazon Stock Snaps 9-Week Losing Streak: Retai...,https://newsable.asianetnews.com/markets/amazo...,"Stocktwits users turned bullish, though Trump'...",Asianet Newsable,04/14/2025
